In [1]:
from ABDB import database as db
import json
import re

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [40]:
def get_paired_sequences_from_sabdab(pdb):
    seqs = dict()
    p = db.fetch(pdb)
    if p is not None:
        for fab in p.fabs:
            numb = fab.get_numbering()
            seqs[pdb + "_" + "".join(numb.keys())] = {x:"".join(numb[x].values()) for x in numb}
        return seqs
    return None

In [2]:
a = False
b = False

c = True

if (a or b) and c:
    print('hi')

In [55]:
def get_paired_sequences_from_sabdab(pdb):
    seqs = dict()
    n = 1
    p = db.fetch(pdb)
    if p is not None:
        for fab in p.fabs:
            numb = fab.get_numbering()
            seqs['H'+str(n)] = "".join(numb['H'].values())
            try:
                seqs['L'+str(n)] = "".join(numb['L'].values())
            except KeyError:
                return None
            n+=1
        return seqs
    return None

In [7]:
with open('data/AB_unpaired.json', 'r') as infile:
    protein_entries = json.load(infile)

In [107]:
pdb_id_regex =  r"""
                (
                    [1-9]  # first character of PDB is 1-9
                    [A-Z0-9]{3}  # remaining 3 characters of PDB are alphanumeric
                )
                [^A-Z0-9]  # no alphanumeric character after, but possibly _ for chain
                """

pattern = re.compile(pdb_id_regex, flags=re.IGNORECASE | re.VERBOSE)

def get_chains_from_sabdab(entry):

    seq = entry['GBSeq_sequence']
    chain = entry['chain']
    id = entry['GBSeq_locus']

    if pattern.match(id):

        pdb_id = id.split('_')[0]
        p = db.fetch(pdb_id)

        if p is not None:
        
            if chain == 'H':
                for fab in p.fabs:
                    numb = fab.get_numbering()
                    if "".join(numb['H'].values()) in seq.upper():

                        try:
                            H = "".join(numb['H'].values())
                            L = "".join(numb['L'].values())
                            return (H, L)
                        except KeyError:
                            return None


            if chain == 'L':
                for fab in p.fabs:
                    numb = fab.get_numbering()
                    if "".join(numb['L'].values()) in seq.upper():

                        try:
                            H = "".join(numb['H'].values())
                            L = "".join(numb['L'].values())
                            return (H, L)
                        except KeyError:
                            return None

        else:
            return None
    else:
        return None

def get_paired_sequences_from_sabdab():
    pdb_pairings = []
    unparied = []
    
    for group in protein_entries:
        unparied_g = []
        for entry in group:
            result = get_chains_from_sabdab(entry)
            if result is not None:
                entry['pdb_hc'] = result[0]
                entry['pdb_lc'] = result[1]
                pdb_pairings.append(entry)
    
            else:
                unparied_g.append(entry)
    
        if unparied_g:
            unparied.append(unparied_g)

def pair_vh_vl_pdb():
    heavy_chains = []
    light_chains = []
    for pairing in pdb_pairings:
        if pairing['chain'] == 'H':
            heavy_chains.append(pairing)
        elif pairing['chain'] == 'L':
            light_chains.append(pairing)
    
    paired_list = []
    unpaired_hcs = []


    for hc in heavy_chains:
        paired = False
                    
        target_lc = hc['pdb_lc']
    
    
        for lc in light_chains:                   
            target_hc = lc['pdb_hc']
    
            if (target_lc in lc['GBSeq_sequence'].upper() and
                    target_hc in hc['GBSeq_sequence'].upper()):
                
                paired_list.append({'heavy_chain': hc,
                               'light_chain': lc})
    
                paired = True                    
                light_chains.remove(lc)           
                break
            
            
        if not paired:
            unpaired_hcs.append(hc)    

    for unpaired_hc in unpaired_hcs:
        paired_list.append({'heavy_chain': unpaired_hc,
                            'light_chain': {'GBSeq_sequence': unpaired_hc['pdb_lc']}})

    for unpaired_lc in light_chains:
        paired_list.append({'heavy_chain': {'GBSeq_sequence': unpaired_lc['pdb_hc']},
                            'light_chain': unpaired_lc})

            


14

In [150]:
len(paired_list)

12

In [151]:
len(unpaired_hcs)

2

In [152]:
len(light_chains)

0

In [154]:
len(paired_list)

14

In [156]:
paired_list[-1]

{'heavy_chain': {'GBSeq_locus': '2GHW_B',
  'GBSeq_moltype': 'AA',
  'GBSeq_update-date': '01-DEC-2020',
  'GBSeq_create-date': '25-OCT-2006',
  'GBSeq_definition': 'Chain B, anti-sars scFv antibody, 80R',
  'GBSeq_accession-version': '2GHW_B',
  'GBSeq_source': 'Homo sapiens (human)',
  'GBSeq_sequence': 'maevqlvqsgggvvqpgkslrlscaasgfafssyamhwvrqapgkglewvavisydgsnkyyadsvkgrftisrdnskntlylqmnslraedtavyycardrsyyldywgqgtlvtvssggggsggggsggggsettltqspatlslspgeratlscrasqsvrsnlawyqqkpgqaprpliydastratgipdrfsgsgsgtdftltisrlepedfavyycqqrsnwpptfgqgtkvevksglvpr',
  'GBSeq_references': [{'GBReference_authors': ['Hwang,W.C.',
     'Lin,Y.',
     'Santelli,E.',
     'Sui,J.',
     'Jaroszewski,L.',
     'Stec,B.',
     'Farzan,M.',
     'Marasco,W.A.',
     'Liddington,R.C.'],
    'GBReference_title': 'Structural basis of neutralization by a human anti-severe acute respiratory syndrome spike protein antibody, 80R',
    'GBReference_journal': 'J. Biol. Chem. 281 (45), 34610-34616 (2006)',
    'GBRefer

In [45]:



bool(pattern.match(string))

False